In [2]:
import torch

创建一个张量，设置`requires_grad=True`来跟踪与它相关的计算

In [3]:
x = torch.ones(2, 2, requires_grad=True)
x

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

针对此张量做一个操作

In [4]:
y = x + 2
y

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)

y作为操作结果被创建，所以它有`grad_fn`

In [6]:
y.grad_fn

针对`y`做更多的操作

In [7]:
z = y * y * 3
z

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)

In [8]:
out = z.mean()
out

tensor(27., grad_fn=<MeanBackward0>)

我们知道后向传播（Backward propogation）是用来算梯度的，那我们以上面的`out`为例来看一下梯度的计算结果。由于`out`这个输出只是一个标量，以下的`out.backward()`等同于`out.backward(torch.tensor(1.))`。

In [9]:
out.backward()

打印`out`关于`x`的梯度

In [10]:
x.grad

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])

由于`out=1/4*sum_{i=1, j=1}^{2, 2} 3*(x_{i,j} + 2)^2`,所以以上的`\partial out/ \partial x_{i,j}`的计算结果正确

`.requires_grad_()`会改变张量的`requires_grad`标记。如果没有提供相应参数，则默认为`False`。

In [14]:
a = torch.randn(2, 2)
a = (a * 3) / (a - 1)
a.requires_grad

False

In [15]:
a.requires_grad_(True)
a.requires_grad

True

In [17]:
b = (a * a).sum()
b.requires_grad, b.grad_fn

(True, <SumBackward0 at 0x1de0d3b7cc8>)

接下来看一个雅可比向量积的例子

In [21]:
x = torch.randn(3, requires_grad=True)
x

tensor([0.3374, 1.8801, 2.3572], requires_grad=True)

In [22]:
y = x * 2
y

tensor([0.6748, 3.7603, 4.7143], grad_fn=<MulBackward0>)

In [23]:
while y.data.norm()<1000:
    y = y * 2

y

tensor([ 172.7489,  962.6292, 1206.8728], grad_fn=<MulBackward0>)

现在，`y`不再是一个标量。`torch.autograd`不能够直接计算整个雅可比，但是我们可以计算雅可比向量积（见与本程序相关的md文档），只需要简单地传递向量给backward()作为参数。

In [24]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)
x.grad

tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])

可以通过将代码包裹在`with torch.no_grad()`，来停止对从跟踪历史中的`.requires_grad=True`的张量自动求导。

In [25]:
print(x.requires_grad)
print((x ** 2).requires_grad)

True
True


In [26]:
with torch.no_grad():
    print((x ** 2).requires_grad)

False
